# Project Description

Write a solution to report the sum of all total investment values in 2016 (`tiv_2016`), for all policyholders who:

- Have the same `tiv_2015` value as one or more other policyholders, and
- Are not located in the same city as any other policyholder (i.e., the (`lat`, `lon`) attribute pairs must be unique).

Round `tiv_2016` to two decimal places.

## Example

### Input:

**Insurance table:**
| pid | tiv_2015 | tiv_2016 | lat | lon |
|-----|----------|----------|-----|-----|
| 1   | 10       | 5        | 10  | 10  |
| 2   | 20       | 20       | 20  | 20  |
| 3   | 10       | 30       | 20  | 20  |
| 4   | 10       | 40       | 40  | 40  |

### Output:
| tiv_2016 |
|----------|
| 45.00    |

**Explanation:**
Only policyholder with `pid` 4 matches the criteria (sharing `tiv_2015` but having a unique `lat`, `lon` pair).

# Intuition
The task requires summing `tiv_2016` values for policyholders who share `tiv_2015` with others but have unique `lat` and `lon`. My first thought is to use window functions to count occurrences of these attributes.

# Approach
I'll use a Common Table Expression (CTE) to calculate the count of `tiv_2015` and `lat`, `lon` pairs for each row. Then, I'll filter for rows where `tiv_2015` is shared (`tiv_count > 1`) and the location is unique (`loc_count = 1`), before summing `tiv_2016`.

# Complexity
- **Time complexity**: $$O(n \log n)$$ due to potential sorting in window functions, although actual performance might be closer to linear with database optimization.
- **Space complexity**: $$O(n)$$ for storing the window function results for each row.

# Code

```sql
WITH cte AS (
    SELECT *,
           COUNT(*) OVER(PARTITION BY tiv_2015) AS tiv_count,
           COUNT(*) OVER(PARTITION BY lat,lon) AS loc_count
    FROM Insurance
)
SELECT ROUND(SUM(tiv_2016),2) AS tiv_2016
FROM cte
WHERE tiv_count > 1 AND loc_count = 1

```

# Method 2 using subqueries

```sql
SELECT ROUND(SUM(tiv_2016), 2) AS tiv_2016
FROM Insurance
WHERE tiv_2015 IN (
    SELECT tiv_2015
    FROM Insurance
    GROUP BY tiv_2015
    HAVING COUNT(*) > 1
)
AND (lat, lon) IN (
    SELECT lat, lon
    FROM Insurance
    GROUP BY lat, lon
    HAVING COUNT(*) = 1
)
```

# Complexity
- **Time complexity**: $$O(n^2)$$ in worst case due to multiple scans and subqueries.
- **Space complexity**: O(n) Potentially less space since it doesn't store all intermediate results.

# Conclusion
Use Method 1 if you require efficiency, but 2 if space is a concern.